In [1]:
import regex as re
import pandas as pd
from nltk.translate.bleu_score import SmoothingFunction
import random
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
validation = dataset['validation']

random.seed(42)

import pandas as pd
import regex as re
from tqdm import tqdm
import nltk
import RagNRollQA

c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
len(validation['question'])

10570

In [3]:
# get length of questions which start with when and doesn't contain what, who, where, how and save the fake when questions indicies
when_questions = []
fake_when_questions = []
fake_when_questions_indices = []
for index, question in enumerate(validation['question']):
    if question.startswith("When") and not any(word in question.lower() for word in [", what", ", who", ", where", ", how"]):
        when_questions.append(question)
    elif question.startswith("When"):
        fake_when_questions.append(question)
        fake_when_questions_indices.append(index)
        
    
        
# remove rows from validation which contains questions inside fake_when_questions_indices
validation = validation.filter(lambda example, idx: idx not in fake_when_questions_indices, with_indices=True)
fake_when_questions_indices

[260,
 1420,
 5063,
 5788,
 6600,
 6808,
 6885,
 6904,
 7362,
 7459,
 8962,
 9035,
 9599,
 9667,
 9877,
 10253,
 10403,
 10422]

In [4]:
len(validation['question'])

10552

In [5]:

def QuestionStartsWith_Accuracy(dataset, startsWith, RagQA):

    correct = 0
    EM = 0
    BLEU = 0
    BLEU1 = 0
    BLEU2 = 0
    BLEU3 = 0
    BLEU4 = 0
    total = 0
    errors = []
    corrects = []
    empties = []
    kolo = 0
    for item in tqdm(dataset):
        random_number = random.randint(0, len(dataset))
        kolo += 1
        included_question = False
        try:
            context = item['context']
            context = re.sub(' +', ' ', context)
            
            question = item['question']
            tempQuestion = question.lower()
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0]
            title = item['title']
    
            # check if question starts with startsWith
            for start in startsWith:
                if tempQuestion.startswith(start):
                    included_question = True
                    break
            
            if included_question:
    
                total += 1
                if total == 74:
                    pass
                outputAnswer = RagQA.answer_question(question, context)
                if outputAnswer == "":
                    empties.append(kolo)
                    outputAnswer = "No_Answer_Found"
                
                # print("Question: " , question)
                # print("Answer: ", answer, "-------" , "Our Answer: ", outputAnswer)
                if outputAnswer in answer or answer in outputAnswer:
                    correct += 1
                    corrects.append(kolo)
                else:
                    # write to when_wrong_answers.txt
                    with open("results/where_wrong_answers.txt", "a") as file:
                        file.write(f"Question: {question}\n")
                        file.write(f"Answer: {answer}\n")
                        file.write(f"Our Answer: {outputAnswer}\n\n")
                        
                        
                if outputAnswer == answer:
                    EM += 1

                n = min(len(outputAnswer.split()), 4)
                if n == 0:
                    BLEUscore = 0
                else:
                    weights = [1.0/n]*n
                    smoothie = SmoothingFunction().method4
                    BLEUscore = nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=weights, smoothing_function=smoothie)
                    
                    BLEU += BLEUscore
                    BLEU1 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(1, 0, 0, 0), smoothing_function=smoothie)
                    BLEU2 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
                    BLEU3 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
                    BLEU4 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
                    

            if kolo % 1000 == 0:
                print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
                print(f"EM: {EM}, out of {total}: {100*EM/total}%")
                print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
                print(f"BLEU1: {BLEU1}, out of {total}: {100*BLEU1/total}%")
                print(f"BLEU2: {BLEU2}, out of {total}: {100*BLEU2/total}%")
                print(f"BLEU3: {BLEU3}, out of {total}: {100*BLEU3/total}%")
                print(f"BLEU4: {BLEU4}, out of {total}: {100*BLEU4/total}%")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Answer: ", answer)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
        print(f"EM: {EM}, out of {total}: {100*EM/total}%")
        print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
        print(f"BLEU1: {BLEU1}, out of {total}: {100*BLEU1/total}%")
        print(f"BLEU2: {BLEU2}, out of {total}: {100*BLEU2/total}%")
        print(f"BLEU3: {BLEU3}, out of {total}: {100*BLEU3/total}%")
        print(f"BLEU4: {BLEU4}, out of {total}: {100*BLEU4/total}%")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    print("Empties: ", len(empties), empties)
    return correct, total, errors, corrects


if __name__ == "__main__":

    startsWith = ["who "]
    RagQA = RagNRollQA.RagNRollQA()
    x = QuestionStartsWith_Accuracy(validation, startsWith, RagQA)

c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want

No facts found in the question


  9%|▊         | 916/10552 [02:26<13:45, 11.67it/s]  c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
  9%|▉         | 997/10552 [02:29<05:08, 31.01it/s]

Correct: 51, out of 203: 25.123152709359605%
EM: 26, out of 203: 12.807881773399014%
BLEU: 50.00304383522701, out of 203: 24.632041298141385%
BLEU1: 71.33977059659844, out of 203: 35.142744136255395%
BLEU2: 50.778041439330195, out of 203: 25.013813516911423%
BLEU3: 44.35792509178475, out of 203: 21.851194626494948%
BLEU4: 40.15586527347335, out of 203: 19.781214420430224%


 10%|▉         | 1045/10552 [02:32<07:13, 21.94it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 10%|█         | 1060/10552 [02:32<06:45, 23.38it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 19%|█▉        | 1988/10552 [03:46<13:17, 10.73it/s]  

Correct: 90, out of 319: 28.213166144200628%
EM: 45, out of 319: 14.106583072100314%
BLEU: 83.7430655795856, out of 319: 26.25174469579486%
BLEU1: 115.24605049849315, out of 319: 36.12728855752136%
BLEU2: 85.51357538701217, out of 319: 26.806763444204442%
BLEU3: 75.89517057095965, out of 319: 23.791589520677007%
BLEU4: 69.94025369815878, out of 319: 21.924844419485513%


 24%|██▎       | 2484/10552 [04:17<05:15, 25.57it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 28%|██▊       | 2990/10552 [04:48<02:57, 42.64it/s]

Correct: 122, out of 396: 30.80808080808081%
EM: 61, out of 396: 15.404040404040405%
BLEU: 110.36314480184244, out of 396: 27.869481010566275%
BLEU1: 146.85377564690822, out of 396: 37.08428677952228%
BLEU2: 112.76754149562342, out of 396: 28.476651892834198%
BLEU3: 101.38411257071881, out of 396: 25.602048628969396%
BLEU4: 93.6577437000364, out of 396: 23.650945378797072%


 32%|███▏      | 3374/10552 [05:09<12:42,  9.42it/s]

In [ ]:
# [174, 315, 326, 349, 472, 488, 493, 498, 530, 667]
# when: 48%